## Setup

In [1]:
# Import packages
import pandas as pd
from ds_common_utils.aux.io.snowflake_tools import SnowflakeTools
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Setup connection
con = SnowflakeTools(
    role="INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD",
    warehouse="INSIGHT_ANALYST_WH",
    database="BDWPRD_DE",
    schema="IA_MERCH_DE"
)

In [3]:
# Date range
start_date = "'2024-06-01'"
end_date = "'2025-05-31'"

## Data

In [4]:
# Set item range table name
table_GM = 'GM_item_range' + datetime.today().strftime('%Y_%m_%d')

In [5]:
# Create temp item table
con.execute_statement_from_sql_file(
    file='sql/0-item-range.sql',
    formatting={'table_name': table_GM,
                'filter' : "AND item_sub_department_name = '400 GROWING MEDIA' "})

snowflake_tools - 2025-07-03 09:30:22.586885+10:00 - Generating new token with 'INSIGHT_ANALYST_MERCH_DE_GENERAL_PRD' role and 'SESSION:ROLE-ANY' scope...
snowflake_tools - 2025-07-03 09:30:27.961793+10:00 - Saved token with '2025-07-03 13:30:27+10:00' expiry...


## Demographic Segments (AU)

In [6]:
GM_ds = con.read_sql_file_into_pandas(
    file="sql/1-demographic-segments.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'AU' 
    }
)

In [7]:
# Check data
GM_ds

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 COIR MULCH BLOCK HYDRO,Homeowner older kids,100865.04,5330.0,2676,1155603.63,60511.0,30355,12658140.01,1480801.0,...,2592008,0.087283,0.088083,0.088157,0.078773,0.079877,0.078463,1.108041,1.102734,1.123549
1,500 COIR MULCH BLOCK HYDRO,Homeowner younger kids,180888.60,9513.0,4871,1155603.63,60511.0,30355,21650028.46,2562825.0,...,2592008,0.156532,0.157211,0.160468,0.134730,0.138243,0.140918,1.161817,1.137205,1.138733
2,500 COIR MULCH BLOCK HYDRO,Older homeowner,543161.02,27693.0,13290,1155603.63,60511.0,30355,87747275.73,9991909.0,...,2592008,0.470024,0.457652,0.437819,0.546059,0.538981,0.487313,0.860756,0.849106,0.898435
3,500 COIR MULCH BLOCK HYDRO,Renter no kids,84448.50,4727.0,2865,1155603.63,60511.0,30355,9191033.47,1092667.0,...,2592008,0.073077,0.078118,0.094383,0.057197,0.058940,0.104600,1.277652,1.325373,0.902323
4,500 COIR MULCH BLOCK HYDRO,Renter with kids,42079.39,2406.0,1506,1155603.63,60511.0,30355,5073012.15,628536.0,...,2592008,0.036413,0.039761,0.049613,0.031570,0.033904,0.062439,1.153423,1.172751,0.794584
5,500 COIR MULCH BLOCK HYDRO,Unclassifiable,8000.41,411.0,228,1155603.63,60511.0,30355,1434624.35,166372.0,...,2592008,0.006923,0.006792,0.007511,0.008928,0.008974,0.009444,0.775459,0.756836,0.795320
6,500 COIR MULCH BLOCK HYDRO,Younger homeowner,196160.67,10431.0,4919,1155603.63,60511.0,30355,22937776.83,2615396.0,...,2592008,0.169747,0.172382,0.162049,0.142744,0.141079,0.116823,1.189175,1.221881,1.387133
7,500 COIR POTTING MIX BLOCK,Homeowner older kids,55588.85,6534.0,3930,708514.12,84519.0,49384,12658140.01,1480801.0,...,2592008,0.078458,0.077308,0.079580,0.078773,0.079877,0.078463,0.996009,0.967838,1.014236
8,500 COIR POTTING MIX BLOCK,Homeowner younger kids,97895.37,11745.0,7133,708514.12,84519.0,49384,21650028.46,2562825.0,...,2592008,0.138170,0.138963,0.144439,0.134730,0.138243,0.140918,1.025532,1.005205,1.024986
9,500 COIR POTTING MIX BLOCK,Older homeowner,343730.03,40950.0,22601,708514.12,84519.0,49384,87747275.73,9991909.0,...,2592008,0.485142,0.484506,0.457658,0.546059,0.538981,0.487313,0.888442,0.898930,0.939146


In [11]:
total_summary = GM_ds.groupby('DEMOGRAPHIC_SEGMENT').agg(TOTAL_SALES_ALL_CLASS=('SEGMENT_SALES', 'first')).reset_index()
total_summary

,DEMOGRAPHIC_SEGMENT,TOTAL_SALES_ALL_CLASS
0,Homeowner older kids,100865.04
1,Homeowner younger kids,180888.60
2,Older homeowner,543161.02
3,Renter no kids,84448.50
4,Renter with kids,42079.39
5,Unclassifiable,8000.41
6,Younger homeowner,196160.67


In [12]:
segment_index_summary = GM_ds[['ITEM_CLASS_NAME', 'DEMOGRAPHIC_SEGMENT', 'SALES_INDEX', 'SEGMENT_SALES']]
segment_index_summary

,ITEM_CLASS_NAME,DEMOGRAPHIC_SEGMENT,SALES_INDEX,SEGMENT_SALES
0,500 COIR MULCH BLOCK HYDRO,Homeowner older kids,1.108041,100865.04
1,500 COIR MULCH BLOCK HYDRO,Homeowner younger kids,1.161817,180888.60
2,500 COIR MULCH BLOCK HYDRO,Older homeowner,0.860756,543161.02
3,500 COIR MULCH BLOCK HYDRO,Renter no kids,1.277652,84448.50
4,500 COIR MULCH BLOCK HYDRO,Renter with kids,1.153423,42079.39
5,500 COIR MULCH BLOCK HYDRO,Unclassifiable,0.775459,8000.41
6,500 COIR MULCH BLOCK HYDRO,Younger homeowner,1.189175,196160.67
7,500 COIR POTTING MIX BLOCK,Homeowner older kids,0.996009,55588.85
8,500 COIR POTTING MIX BLOCK,Homeowner younger kids,1.025532,97895.37
9,500 COIR POTTING MIX BLOCK,Older homeowner,0.888442,343730.03


## Commercial Industry Segment (NZ)

In [8]:
# Get data
GM_is = con.read_sql_file_into_pandas(
    file="sql/5-commercial-industry-segment.sql",
    formatting={
        'start_date': start_date,
        'end_date': end_date,
        'n_commercial_segments': 10, 
        'table_name': table_GM,
        'level' : 'item_class_name',
        'country' : 'NZ' 
    }
)

In [9]:
# Check data
GM_is.sort_values(by='SALES_INDEX', ascending=False)

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SEGMENT_SALES,SEGMENT_QUANTITY,SEGMENT_CUSTOMERS,TOTAL_SALES,TOTAL_QUANTITY,TOTAL_CUSTOMERS,TOTAL_SEG_SALES,TOTAL_SEG_QUANTITY,...,OVERALL_CUSTOMERS,SEGMENT_SALES_SHARE,SEGMENT_QUANTITY_SHARE,SEGMENT_CUSTOMER_SHARE,OVERALL_SALES_SHARE,OVERALL_QUANTITY_SHARE,OVERALL_CUSTOMER_SHARE,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
2,500 COIR POTTING MIX BLOCK,Cleaning Services,516.18,90.0,21,12144.87,2015.0,427,62361.57,7410.0,...,20638,0.042502,0.044665,0.049180,0.028692,0.029172,0.036583,1.481292,1.531070,1.344340
23,500 MULCHES,Commercial Builder,14089.75,1543.0,167,374642.93,39672.0,4307,60169.97,6963.0,...,20638,0.037608,0.038894,0.038774,0.027684,0.027413,0.030381,1.358486,1.418832,1.276258
0,500 COIR POTTING MIX BLOCK,Accommodation and Food Services,999.10,144.0,49,12144.87,2015.0,427,133972.43,15573.0,...,20638,0.082265,0.071464,0.114754,0.061640,0.061309,0.084117,1.334596,1.165630,1.364219
22,500 MULCHES,Cleaning Services,14113.21,1574.0,155,374642.93,39672.0,4307,62361.57,7410.0,...,20638,0.037671,0.039675,0.035988,0.028692,0.029172,0.036583,1.312927,1.360029,0.983736
21,500 MULCHES,Carpentry Services,7108.71,410.0,69,374642.93,39672.0,4307,32410.63,3542.0,...,20638,0.018975,0.010335,0.016020,0.014912,0.013944,0.020399,1.272433,0.741134,0.785333
5,500 COIR POTTING MIX BLOCK,Landscaper and Gardening Services,1362.89,226.0,48,12144.87,2015.0,427,206119.60,23571.0,...,20638,0.112219,0.112159,0.112412,0.094835,0.092797,0.074087,1.183308,1.208651,1.517297
14,500 COMPOSTS SOIL CONDITIONERS,Electrical Services,31326.59,4279.0,739,677382.24,94704.0,10977,85934.88,10116.0,...,20638,0.046247,0.045183,0.067323,0.039538,0.039826,0.063960,1.169659,1.134517,1.052580
8,500 COIR POTTING MIX BLOCK,Residential Builder,1999.29,444.0,64,12144.87,2015.0,427,309412.17,35919.0,...,20638,0.164620,0.220347,0.149883,0.142360,0.141409,0.183690,1.156365,1.558222,0.815956
30,500 POTTING MIX SMALL BAG,Accommodation and Food Services,7633.47,854.0,359,109123.87,12266.0,4399,133972.43,15573.0,...,20638,0.069952,0.069623,0.081609,0.061640,0.061309,0.084117,1.134844,1.135608,0.970184
37,500 POTTING MIX SMALL BAG,Professional Computer and Scientific Services,38067.76,4321.0,915,109123.87,12266.0,4399,676139.59,79072.0,...,20638,0.348849,0.352275,0.208002,0.311091,0.311299,0.206755,1.121374,1.131630,1.006031


In [13]:
total_summary = GM_is.groupby('ITEM_CLASS_NAME').agg(
    TOTAL_SALES_ALL_SEGMENTS=('TOTAL_SALES', 'first'),  # consistent within class
    TOTAL_QUANTITY_ALL_SEGMENTS=('TOTAL_QUANTITY', 'first')
).reset_index()
total_summary

,ITEM_CLASS_NAME,TOTAL_SALES_ALL_SEGMENTS,TOTAL_QUANTITY_ALL_SEGMENTS
0,500 COIR POTTING MIX BLOCK,12144.87,2015.0
1,500 COMPOSTS SOIL CONDITIONERS,677382.24,94704.0
2,500 MULCHES,374642.93,39672.0
3,500 POTTING MIX SMALL BAG,109123.87,12266.0
4,500 POTTING MIXES,1000154.30,105350.0


In [14]:
segment_index_summary = GM_is[['ITEM_CLASS_NAME', 'INDUSTRY_SEGMENT', 'SALES_INDEX', 'QUANTITY_INDEX', 'CUSTOMER_INDEX']]
segment_index_summary

,ITEM_CLASS_NAME,INDUSTRY_SEGMENT,SALES_INDEX,QUANTITY_INDEX,CUSTOMER_INDEX
0,500 COIR POTTING MIX BLOCK,Accommodation and Food Services,1.334596,1.165630,1.364219
1,500 COIR POTTING MIX BLOCK,Carpentry Services,0.601750,0.782969,0.574048
2,500 COIR POTTING MIX BLOCK,Cleaning Services,1.481292,1.531070,1.344340
3,500 COIR POTTING MIX BLOCK,Commercial Builder,0.349295,0.380184,0.925019
4,500 COIR POTTING MIX BLOCK,Electrical Services,1.116412,1.208742,0.952001
5,500 COIR POTTING MIX BLOCK,Landscaper and Gardening Services,1.183308,1.208651,1.517297
6,500 COIR POTTING MIX BLOCK,Manufacturing,1.119935,1.007800,1.344390
7,500 COIR POTTING MIX BLOCK,Professional Computer and Scientific Services,0.793175,0.734935,0.860855
8,500 COIR POTTING MIX BLOCK,Residential Builder,1.156365,1.558222,0.815956
9,500 COIR POTTING MIX BLOCK,Retail and Wholesale Trade,0.952994,0.788225,0.870765


## Drop item range table

In [10]:
# Drop item range table
con.execute_statement_from_sql_string(
    statement='DROP TABLE bdwprd_de.ia_merch_de.{table_name};',
    formatting={'table_name': table_GM}
    )